# 🤖 MODELAGEM DE MACHINE LEARNING PARA SCORE DE CRÉDITO

**Objetivo:** Desenvolver, avaliar e selecionar modelos de Machine Learning para prever a inadimplência de clientes.

### 📌 Etapas
- Reparar dados para Machine Learning
- Codificar variáveis categóricas
- Dividir em treino e teste
- Treinar múltiplos modelos de classificação
- Avaliar performance com métricas adequadas
- Selecionar melhor modelo
- Analisar feature importance
- Salvar modelo para produção

### 🏆 Modelo que vamos testar
- Random Forest (Ensemble)
- KNN (K-Nearest Neighbors)
- Logistic Regression (Linear)
- Decision-Tree (Base do Random Forest)

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # Para salvar modelos
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, ConfusionMatrixDisplay
)

# Configs de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
%matplotlib inline

In [ ]:
# Carregando dados do SQLite
DB_PATH = 'data/database.db'
print(f"\n📂 Carregando dados: {DB_PATH}")

conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM clientes", conn)
conn.close()

In [ ]:
# Codificando variáveis categóricas
print("🔄 CODIFICANDO VARIÁVEIS CATEGÓRICAS")

# Identifica
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'inadimplente' in cat_cols:
  cat_cols.remove('inadimplente')

print(f"\n📝 Variáveis categóricas: {cat_cols}")

# Codifica e salva encoders
encoders = {}

os.makedirs('../models', exist_ok=True)

for col in cat_cols:
  encoder = LabelEncoder()
  df[col] = encoder.fit_transform(df[col])
  encoders[col] = encoder
    
  joblib.dump(encoder, f'../models/encoder_{col}.pkl')
  print(f"   ✅ {col}: {list(encoder.classes_)}")

print(f"\n💾 {len(encoders)} encoders salvos em ../models/")